In [3]:
import pandas as pd
import requests 
import lxml.html
import os


In [4]:
# Get All Events
# https://eurovision.tv/event/tel-aviv-2019/grand-final/results/albania

from fake_useragent import UserAgent
ua = UserAgent()

url = 'https://eurovision.tv/events/'
r = requests.get( url , headers = {'User-Agent': ua.random}  )
html = lxml.html.fromstring( r.text )
events = html.xpath('//div[@class="flex flex-wrap"]/div/div/a')

event_links = [i.get('href') for i in events]
df_events = pd.DataFrame( event_links , columns= ['url'])

df_events['year'] = [ int(r.split('-')[ len(r.split('-'))-1 ]) for i,r in df_events['url'].iteritems() ]
df_events['place'] = [ r.split('/')[ len(r.split('/'))-1 ] for i,r in df_events['url'].iteritems() ]
df_events['place'] = [ " ".join(r.split('-')[ :len(r.split('-'))-1 ]) for i,r in df_events['place'].iteritems() ]

df_events

,url,year,place
0,https://eurovision.tv/event/rotterdam-2021,2021,rotterdam
1,https://eurovision.tv/event/rotterdam-2020,2020,rotterdam
2,https://eurovision.tv/event/tel-aviv-2019,2019,tel aviv
3,https://eurovision.tv/event/lisbon-2018,2018,lisbon
4,https://eurovision.tv/event/kyiv-2017,2017,kyiv
...,...,...,...
61,https://eurovision.tv/event/london-1960,1960,london
62,https://eurovision.tv/event/cannes-1959,1959,cannes
63,https://eurovision.tv/event/hilversum-1958,1958,hilversum
64,https://eurovision.tv/event/frankfurt-1957,1957,frankfurt


In [47]:
import time
import random
#Extract Data

def get_pts( df , year ): #Get given years votes per country

    from fake_useragent import UserAgent
    ua = UserAgent()

    all_pts = pd.DataFrame()
    #Format changes in 2004 - Istanbul

    if year < 2004: url = df[ df['year']==year ]['url'].tolist()[0] + '/final/results/'
    if year >= 2004: url = df[ df['year']==year ]['url'].tolist()[0] + '/grand-final/results/'
    #Get all countries for results
    r = requests.get( url , headers = {'User-Agent': ua.random}  )
    html = lxml.html.fromstring( r.text )
    countries = html.xpath('//option')
    country_links = [i.get('value') for i in countries] 

    proxies = {
        'http': 'http://135.181.68.88:3128',
        'https': 'http://135.181.68.88:3128',
    }

    for c in country_links: #Get points received
        if c:
            try:
                r = requests.get( c ,
                    headers = {'User-Agent': ua.random} ,
                    #proxies= proxies
                    )
                pts = pd.read_html( r.text )[0]
                pts.columns = ['pts','pts_from']
                pts['year'] = year
                pts['points_to'] = c.split('/')[len(c.split('/'))-1] 
                all_pts = all_pts.append( pts )
            except:
                print( c )
        time.sleep( random.random()*3 )
    if len(all_pts) == 0: print( str(year), 'is empty' )
    return all_pts


In [59]:
year = 2018
folder = r'C:\Users\csucuogl\Documents\GitHub\IdeasTesting\Songs\data'
temp = get_pts( df_events , year )

temp.to_csv( os.path.join(folder,'results_'+str(year)+'.csv' ) )

temp

,pts,pts_from,year,points_to
0,12,Albania,2018,italy
1,12,Malta,2018,italy
2,12,Germany,2018,italy
3,10,France,2018,italy
4,10,Slovenia,2018,italy
...,...,...,...,...
9,1,United Kingdom,2018,san-marino
0,6,Malta,2018,australia
1,2,Denmark,2018,australia
2,1,United Kingdom,2018,australia
